In [ ]:
%cd ..

In [ ]:
ls

In [ ]:
!nvidia-smi

In [ ]:
import torch
print(torch.cuda.is_available())   # True if GPU is being used
print(torch.cuda.current_device()) # GPU index (0 = first GPU)
print(torch.cuda.get_device_name(0))  # GPU name


In [ ]:
!yolo detect train data = configs/data_3.yaml model = yolov8s.pt epochs = 1 imgsz = 640 batch = 4

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")
model.train(data="configs/data_3.yaml", epochs=1, imgsz=640, batch = 4)


In [ ]:
!yolo detect train data = configs/data_3.yaml model = yolov8s.pt

In [ ]:
!yolo detect train data = configs/data_3.yaml model = yolov8n.pt epochs = 10 imgsz = 416 batch = 4 device = 0 project = results

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(data="configs/data_3.yaml", 
            epochs=15, 
            imgsz=640, 
            batch = 2,
            device = 0 if torch.cuda.is_available() else 'cpu', 
            project = 'results'
           )


In [ ]:
!yolo detect val model = results/train11/weights/best.pt data = configs/data_3.yaml project = results

In [ ]:
model = YOLO('results/train11/weights/best.pt')
metrics = model.val(data = 'configs/data_3.yaml',
                   project = 'results',
                   name = 'val11')

In [ ]:
!yolo detect val model = results/train11/weights/best.pt data = configs/data_3.yaml split = test project = results name = test11

In [ ]:
!yolo export model=results/train11/weights/best.pt format=onnx project=models name=best_model_11

In [ ]:
model = YOLO('results/train11/weights/best.pt')
model.export(
    format = 'onnx',
    project = 'models',
    name = 'CLI_est_model_11'
)

In [ ]:
ls

In [ ]:
!mv results/train11/weights/best.onnx models/

In [ ]:
import cv2

cap = cv2.VideoCapture("data/vedio.mp4")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, stream=True)
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())
            label = model.names[cls]
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (int(x1), int(y1)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

    cv2.imshow("Video Detections", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
